In [107]:
# Read English Training Data
# By: Benjamin Kulis
import string, math

input_file = open('../Data/Input/LangId.train.English', 'r', encoding = 'Latin1')
lines = input_file.readlines()
input_file.close()

words = []
english_vocab = []

for l in lines:
    for p in string.punctuation:
        l = l.replace(p, "")
    words.append(l.strip().lower())
    
i = 0
while i < len(words):
    english_vocab.extend(words[i].split())
    i += 1

In [108]:
# Create English Bigram
english_currWords = {}
english_prevWord = {}
english_bigram = {}

for i in range(len(english_vocab) - 1):
    if (english_vocab[i], english_vocab[i + 1]) in english_currWords.keys():
        english_currWords[(english_vocab[i], english_vocab[i + 1])] += 1
    else:
        english_currWords[(english_vocab[i], english_vocab[i + 1])] = 1
for i in range(len(english_vocab)):
    if english_vocab[i] in english_prevWord.keys():
        english_prevWord[english_vocab[i]] += 1
    else:
        english_prevWord[english_vocab[i]] = 1
        
for key in english_currWords.keys():
    english_bigram[(key, key[0])] = english_currWords[key] / english_prevWord[key[0]]

In [109]:
# Read French Training Data
input_file = open('../Data/Input/LangId.train.French', 'r', encoding = 'Latin1')
lines = input_file.readlines()
input_file.close()

words = []
french_vocab = []

for l in lines:
    for p in string.punctuation:
        l = l.replace(p, "")
    words.append(l.strip().lower())
    
i = 0
while i < len(words):
    french_vocab.extend(words[i].split())
    i += 1

In [110]:
# Create French Bigram
french_currWords = {}
french_prevWord = {}
french_bigram = {}

for i in range(len(french_vocab) - 1):
    if (french_vocab[i], french_vocab[i + 1]) in french_currWords.keys():
        french_currWords[(french_vocab[i], french_vocab[i + 1])] += 1
    else:
        french_currWords[(french_vocab[i], french_vocab[i + 1])] = 1
for i in range(len(french_vocab)):
    if french_vocab[i] in french_prevWord.keys():
        french_prevWord[french_vocab[i]] += 1
    else:
        french_prevWord[french_vocab[i]] = 1
        
for key in french_currWords.keys():
    french_bigram[(key, key[0])] = french_currWords[key] / french_prevWord[key[0]]

In [111]:
# Read Italian Training Data
input_file = open('../Data/Input/LangId.train.Italian', 'r', encoding = 'Latin1')
lines = input_file.readlines()
input_file.close()

words = []
italian_vocab = []

for l in lines:
    for p in string.punctuation:
        l = l.replace(p, "")
    words.append(l.strip().lower())
    
i = 0
while i < len(words):
    italian_vocab.extend(words[i].split())
    i += 1

In [112]:
# Create Italian Bigram
italian_currWords = {}
italian_prevWord = {}
italian_bigram = {}

for i in range(len(italian_vocab) - 1):
    if (italian_vocab[i], italian_vocab[i + 1]) in italian_currWords.keys():
        italian_currWords[(italian_vocab[i], italian_vocab[i + 1])] += 1
    else:
        italian_currWords[(italian_vocab[i], italian_vocab[i + 1])] = 1
for i in range(len(italian_vocab)):
    if italian_vocab[i] in italian_prevWord.keys():
        italian_prevWord[italian_vocab[i]] += 1
    else:
        italian_prevWord[italian_vocab[i]] = 1
        
for key in italian_currWords.keys():
    italian_bigram[(key, key[0])] = italian_currWords[key] / italian_prevWord[key[0]]

In [113]:
# Create Vocabulary for Test Data
test_file = open('../Data/Validation/LangId.test', 'r', encoding = 'Latin1')
lines = test_file.readlines()
test_file.close()

test_vocab = []
for l in lines:
    words = []
    line_vocab = []
    for p in string.punctuation:
        l = l.replace(p, "")
    words.append(l.strip().lower())

    i = 0
    while i < len(words):
        line_vocab.extend(words[i].split())
        i += 1
    test_vocab.append(line_vocab)

In [114]:
# Test English
english_test_log = []

for line_vocab in test_vocab:
    test_log = 0
    test_currWords = []
    test_prevWord = []
    for i in range(len(line_vocab) - 1):
        test_currWords.append((line_vocab[i], line_vocab[i + 1]))
    for word in line_vocab:
        test_prevWord.append(word)

    # Apply GT smoothing
    # Sets up the bigram and pairs of words and adds the ones that havent been seen
    for word in test_currWords:
        if word[0] not in english_prevWord.keys():
            english_prevWord[word[0]] = 0
        if word not in english_currWords.keys():
            english_currWords[word] = 0
        if (word, word[0]) not in english_bigram.keys():
            english_bigram[(word, word[0])] = 0

    total_distinct_words = len(english_prevWord.keys())
    possible_word_pairs = total_distinct_words * total_distinct_words
    seen_distinct_pairs = len(english_currWords.keys())

    count_appearances = {}
    for key in english_currWords.keys():
        if english_currWords[key] in count_appearances.keys():
            count_appearances[english_currWords[key]] += 1
        else:
            count_appearances[english_currWords[key]] = 1
    count_appearances[0] = (possible_word_pairs - seen_distinct_pairs)

    sorted_appearances = sorted(count_appearances.keys())

    total_seen_pairs = 0
    for c in count_appearances.keys():
        total_seen_pairs += (c * count_appearances[c])

    # Calculate C star
    count = 0
    cStarDict = {}
    for c in range(len(sorted_appearances)):
        if count >= len(sorted_appearances) - 1:
            cStarDict[sorted_appearances[c]] = cStarDict[0]
        if count < len(sorted_appearances) - 1:
            cStarDict[sorted_appearances[c]] = (sorted_appearances[c + 1] * count_appearances[sorted_appearances[c + 1]]) / count_appearances[sorted_appearances[c]]
        count += 1

    # Calculate P star
    pStarDict = {}
    for c in cStarDict.keys():
        pStarDict[c] = (cStarDict[c] * count_appearances[c]) / total_seen_pairs

    # Update bigram probabilities
    for i in english_bigram:
        c = english_currWords[i[0]]
        english_bigram[i] = pStarDict[c] / count_appearances[c]

    for word in range(len(line_vocab) - 1):
        test_log += math.log(english_bigram[((line_vocab[word], line_vocab[word + 1]), (line_vocab[word], line_vocab[word + 1])[0])], 2)
    english_test_log.append(test_log)

In [115]:
# Test French
french_test_log = []

for line_vocab in test_vocab:
    test_log = 0
    test_currWords = []
    test_prevWord = []
    for i in range(len(line_vocab) - 1):
        test_currWords.append((line_vocab[i], line_vocab[i + 1]))
    for word in line_vocab:
        test_prevWord.append(word)

    # Apply GT smoothing
    # Sets up the bigram and pairs of words and adds the ones that havent been seen
    for word in test_currWords:
        if word[0] not in french_prevWord.keys():
            french_prevWord[word[0]] = 0
        if word not in french_currWords.keys():
            french_currWords[word] = 0
        if (word, word[0]) not in french_bigram.keys():
            french_bigram[(word, word[0])] = 0

    total_distinct_words = len(french_prevWord.keys())
    possible_word_pairs = total_distinct_words * total_distinct_words
    seen_distinct_pairs = len(french_currWords.keys())

    count_appearances = {}
    for key in french_currWords.keys():
        if french_currWords[key] in count_appearances.keys():
            count_appearances[french_currWords[key]] += 1
        else:
            count_appearances[french_currWords[key]] = 1
    count_appearances[0] = (possible_word_pairs - seen_distinct_pairs)

    sorted_appearances = sorted(count_appearances.keys())

    total_seen_pairs = 0
    for c in count_appearances.keys():
        total_seen_pairs += (c * count_appearances[c])

    # Calculate C star
    count = 0
    cStarDict = {}
    for c in range(len(sorted_appearances)):
        if count >= len(sorted_appearances) - 1:
            cStarDict[sorted_appearances[c]] = cStarDict[0]
        if count < len(sorted_appearances) - 1:
            cStarDict[sorted_appearances[c]] = (sorted_appearances[c + 1] * count_appearances[sorted_appearances[c + 1]]) / count_appearances[sorted_appearances[c]]
        count += 1

    # Calculate P star
    pStarDict = {}
    for c in cStarDict.keys():
        pStarDict[c] = (cStarDict[c] * count_appearances[c]) / total_seen_pairs

    # Update bigram probabilities
    for i in french_bigram:
        c = french_currWords[i[0]]
        french_bigram[i] = pStarDict[c] / count_appearances[c]

    for word in range(len(line_vocab) - 1):
        test_log += math.log(french_bigram[((line_vocab[word], line_vocab[word + 1]), (line_vocab[word], line_vocab[word + 1])[0])], 2)
    french_test_log.append(test_log)

In [116]:
# Test Italian
italian_test_log = []

for line_vocab in test_vocab:
    test_log = 0
    test_currWords = []
    test_prevWord = []
    for i in range(len(line_vocab) - 1):
        test_currWords.append((line_vocab[i], line_vocab[i + 1]))
    for word in line_vocab:
        test_prevWord.append(word)

    # Apply GT smoothing
    # Sets up the bigram and pairs of words and adds the ones that havent been seen
    for word in test_currWords:
        if word[0] not in italian_prevWord.keys():
            italian_prevWord[word[0]] = 0
        if word not in italian_currWords.keys():
            italian_currWords[word] = 0
        if (word, word[0]) not in italian_bigram.keys():
            italian_bigram[(word, word[0])] = 0

    total_distinct_words = len(italian_prevWord.keys())
    possible_word_pairs = total_distinct_words * total_distinct_words
    seen_distinct_pairs = len(italian_currWords.keys())

    count_appearances = {}
    for key in italian_currWords.keys():
        if italian_currWords[key] in count_appearances.keys():
            count_appearances[italian_currWords[key]] += 1
        else:
            count_appearances[italian_currWords[key]] = 1
    count_appearances[0] = (possible_word_pairs - seen_distinct_pairs)

    sorted_appearances = sorted(count_appearances.keys())

    total_seen_pairs = 0
    for c in count_appearances.keys():
        total_seen_pairs += (c * count_appearances[c])

    # Calculate C star
    count = 0
    cStarDict = {}
    for c in range(len(sorted_appearances)):
        if count >= len(sorted_appearances) - 1:
            cStarDict[sorted_appearances[c]] = cStarDict[0]
        if count < len(sorted_appearances) - 1:
            cStarDict[sorted_appearances[c]] = (sorted_appearances[c + 1] * count_appearances[sorted_appearances[c + 1]]) / count_appearances[sorted_appearances[c]]
        count += 1

    # Calculate P star
    pStarDict = {}
    for c in cStarDict.keys():
        pStarDict[c] = (cStarDict[c] * count_appearances[c]) / total_seen_pairs

    # Update bigram probabilities
    for i in italian_bigram:
        c = italian_currWords[i[0]]
        italian_bigram[i] = pStarDict[c] / count_appearances[c]

    for word in range(len(line_vocab) - 1):
        test_log += math.log(italian_bigram[((line_vocab[word], line_vocab[word + 1]), (line_vocab[word], line_vocab[word + 1])[0])], 2)
    italian_test_log.append(test_log)

In [117]:
# Compare and Write Predictions to Output File
output_file = open('../Data/Output/wordLangId2.out', 'w')

for i in range(len(english_test_log)):
    lang_prediction = max(english_test_log[i], french_test_log[i], italian_test_log[i])

    if lang_prediction == english_test_log[i]:
        output_file.write(str(i + 1) + " English")
    elif lang_prediction == french_test_log[i]:
        output_file.write(str(i + 1) + " French")
    else:
        output_file.write(str(i + 1) + " Italian")

    if (i + 1 < len(english_test_log)):
         output_file.write("\n")
output_file.close()

In [118]:
# Check Prediction Accuracy
prediction_file = open('../Data/Output/wordLangId2.out', 'r')
prediction_lines = prediction_file.readlines()
prediction_file.close()

solutions_file = open('../Data/Validation/labels.sol', 'r')
solution_lines = solutions_file.readlines()
solutions_file.close()

correct = 0
for i in range(len(solution_lines)):
    if prediction_lines[i] == solution_lines[i]:
        correct += 1
print("Prediction Accuracy: " + str((correct/len(solution_lines)) * 100) + "%")

Prediction Accuracy: 99.66666666666667%
